# completions

> Autocomplete functionality
- order: 11

Adapted from the [stata_kernel version](https://github.com/kylebarron/stata_kernel/blob/master/stata_kernel/completions.py).

In [ ]:
#| default_exp completions
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from nbstata.stata import get_global, pwd
from nbstata.stata_session import StataSession
from nbstata.magics import StataMagics
from nbstata.completion_env import CompletionEnv, Env
from fastcore.basics import patch_to
import os
import re
import platform

In [ ]:
#| export
class CompletionsManager():
    def __init__(self, stata_session: StataSession):
        """"""
        self.stata_session = stata_session
        self.available_magics = list(StataMagics.available_magics.keys())
        self.env_helper = CompletionEnv()

In [ ]:
#| hide
from unittest.mock import Mock

In [ ]:
#| hide
test_instance = CompletionsManager(Mock())
test_instance.available_magics

['browse',
 'head',
 'tail',
 'locals',
 'delimit',
 'help',
 'quietly',
 'noecho',
 'echo']

In [ ]:
#| export
@patch_to(CompletionsManager)
def get_globals(self):
    if self.stata_session.suggestions:
        return {k: get_global(k) for k in self.stata_session.suggestions['globals']}
    else:
        return {}

In [ ]:
#| export
@patch_to(CompletionsManager)
def get_file_paths(self, chunk):
    """Get file paths based on chunk
    Args:
        chunk (str): chunk of text after last space. Doesn't include string
            punctuation characters
    Returns:
        (List[str]): folders and files at that location
    """
    # If local exists, return empty list
    if re.search(r'[`\']', chunk):
        return []

    # Define directory separator
    dir_sep = '/'
    if platform.system() == 'Windows':
        if '/' not in chunk:
            dir_sep = '\\'

    # Get directory without ending file, and without / or \
    if any(x in chunk for x in ['/', '\\']):
        ind = max(chunk.rfind('/'), chunk.rfind('\\'))
        user_folder = chunk[:ind + 1]
        user_starts = chunk[ind + 1:]

        # Replace multiple consecutive / with a single /
        user_folder = re.sub(r'/+', '/', user_folder)
        user_folder = re.sub(r'\\+', r'\\', user_folder)

    else:
        user_folder = ''
        user_starts = chunk

    # Replace globals with their values
    globals_re = r'\$\{?((?![0-9_])\w{1,32})\}?'
    try:
        folder = re.sub(
            globals_re, 
            lambda x: self.get_globals()[x.group(1)], 
            user_folder
        )
    except KeyError:
        # If the global doesn't exist (aka it hasn't been defined in 
        # the Stata environment yet), then there are no paths to check
        return []

    # Use Stata's relative path
    abspath = re.search(r'^([/~]|[a-zA-Z]:)', folder)
    if not abspath:
        folder = pwd() + '/' + folder

    try:
        top_dir, dirs, files = next(os.walk(os.path.expanduser(folder)))
        results = [x + dir_sep for x in dirs] + files
        results = [
            user_folder + x for x in results if not x.startswith('.')
            and re.match(re.escape(user_starts), x, re.I)]

    except StopIteration:
        results = []

    return sorted(results)

In [ ]:
#| eval: false
from nbstata.config import launch_stata
from nbstata.stata_more import run_sfi

In [ ]:
#| eval: false
launch_stata(splash=False)
test_stata = StataSession()
test_instance = CompletionsManager(test_stata)
test_instance.get_file_paths("0")

['00_misc_utils.ipynb',
 '01_config.ipynb',
 '02_stata.ipynb',
 '03_stata_more.ipynb',
 '04_code_utils.ipynb',
 '05_noecho.ipynb',
 '06_pandas.ipynb',
 '07_browse.ipynb',
 '08_stata_session.ipynb',
 '09_magics.ipynb']

In [ ]:
#| eval: false
run_sfi('global in_path "../nbstata"')
test_instance.stata_session.refresh_suggestions()
test_instance.get_file_paths("$in_path/com")

['$in_path/completion_env.py', '$in_path/completions.py']

In [ ]:
#| export

relevant_suggestion_keys = {
    Env.NONE: [],
    Env.GENERAL: ['varlist', 'scalars'],
    Env.LOCAL: ['locals'],
    Env.GLOBAL: ['globals'],
    Env.SCALAR: ['scalars'],
    Env.MATRIX: ['matrices'],
    Env.SCALAR_VAR: ['scalars', 'varlist'],
    Env.MATRIX_VAR: ['matrices', 'varlist'],
    Env.STRING: [],
}

@patch_to(CompletionsManager)
def get(self, starts, env, rcomp):
    """Return environment-aware completions list."""
    if env is Env.MAGIC:
        candidate_suggestions = self.available_magics
    else:
        candidate_suggestions = [suggestion
                                 for key in relevant_suggestion_keys[env]
                                 for suggestion in self.stata_session.suggestions[key]]
    relevant_suggestions = [candidate + rcomp 
                            for candidate in candidate_suggestions
                            if candidate.startswith(starts)]
    if env in [Env.GENERAL, Env.STRING]:
        relevant_suggestions += self.get_file_paths(starts)
    return relevant_suggestions

#     elif env == 9:
#         if len(starts) > 1:
#             builtins = [
#                 var for var in mata_builtins if var.startswith(starts)]
#         else:
#             builtins = []

#         if re.search(r'[/\\]', starts):
#             paths = self.get_file_paths(starts)
#         else:
#             paths = []

#         return [
#             var for var in self.stata_session.suggestions['mata']
#             if var.startswith(starts)] + builtins + paths

In [ ]:
#| export
@patch_to(CompletionsManager)
def do(self, code, cursor_pos, sc_delimiter=False):
    if self.stata_session.suggestions is None:
        self.stata_session.refresh_suggestions()
    env, pos, chunk, rcomp = self.env_helper.get_env(
        code[:cursor_pos], 
        code[cursor_pos:(cursor_pos + 2)],
        sc_delimiter,
    )
    return pos, cursor_pos, self.get(chunk, env, rcomp)

In [ ]:
#| eval: False
from fastcore.test import test_eq

In [ ]:
#| eval: False
def completions_test_setup(code):
    global test_instance
    run_sfi("clear all")
    run_sfi(code)
    test_instance.stata_session.clear_suggestions()
    

def _complete(code, cursor_pos):
    _, _, matches = test_instance.do(code, cursor_pos)
    return matches

In [ ]:
#| eval: False
completions_test_setup("gen var1 = 1")
code = "list va"
cursor_pos = 7

test_eq(
    test_instance.env_helper.get_env(
        code[:cursor_pos], code[cursor_pos:(cursor_pos + 2)],
        False),
    (0, 5, 'va', ''),
) 
test_eq(
    _complete(code, cursor_pos),
    ['var1'],
)

In [ ]:
#| hide
#| eval: False
completions_test_setup('')
test_eq(
    _complete("use sideb", 9),
    ["sidebar.yml"],
)

In [ ]:
#| hide
#| eval: False
code = 'use "../manual_test_nbs/delimit t'
test_eq(
    _complete(code, len(code)),
    ["../manual_test_nbs/delimit tests.ipynb"],
)

In [ ]:
#| hide
#| eval: False
completions_test_setup('global indir "../manual_test_nbs"')
code = 'use "$indir/delimit t'
test_eq(
    _complete(code, len(code)),
    ["$indir/delimit tests.ipynb"],
)

In [ ]:
#| eval: False
completions_test_setup('local test_local "test value"')
test_eq(
    _complete("list `t", 7),
    ["test_local'"],
)
run_sfi('local test_local ""')

In [ ]:
#| hide
#| eval: False
test_eq(
    _complete("list `t'", 7),
    ["test_local"],
)
test_eq(
    _complete("list `t'", 8),
    [],
)

In [ ]:
#| eval: False
completions_test_setup('global test_global "test value"')
test_eq(
    _complete("list ${tes}", 10),
    ['test_global'],
)

In [ ]:
#| hide
#| eval: False
test_eq(
    _complete("list $tes", 9),
    ['test_global'],
)
test_eq(
    _complete("list ${tes", 10),
    ['test_global}'],
)

In [ ]:
#| hide
#| eval: False
completions_test_setup('scalar test_scalar = 5')
test_eq(
    _complete("disp scalar(tes", 15),
    ['test_scalar)'],
)
test_eq(
    _complete("disp scalar(tes)", 15),
    ['test_scalar'],
)
test_eq(
    _complete("disp `=scalar(tes", 17),
    ["test_scalar)"],
)
test_eq(
    _complete("disp `=scalar(tes'", 17),
    ["test_scalar)"],
)
test_eq(
    _complete("disp `=tes", 10),
    ["test_scalar"],
)

In [ ]:
#| hide
#| eval: False
completions_test_setup('gen var1 = 1')
test_eq(
    _complete("disp `=va", 10),
    ["var1"],
)

In [ ]:
#| hide
#| eval: False
completions_test_setup(r'matrix test_matrix = (1,2,3\4,5,6)')
test_eq(
    _complete("matrix A = tes", 14),
    ['test_matrix'],
)

In [ ]:
#| hide
#| eval: False
completions_test_setup('gen x1 = 1 \n scalar x2 = 2')
code = """\
#delimit;
scalar
list x"""
test_eq(
    _complete(code, len(code)),
    ['x2'],
)

In [ ]:
#| hide
#| eval: False
code = "*%ec"
test_eq(
    _complete(code, len(code)),
    ["echo"],
)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()